In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import scipy as sp
import sys, os
import pickle
from matplotlib import pyplot as plt
from tenpy.tools.params import asConfig
from mpomps_tenpy import *
from PartonSquare import *

/Users/hk/miniconda3/lib/python3.8/site-packages/tenpy/tools/optimization.py:317: UserWarning: Couldn't load compiled cython code. Code will run a bit slower.
  warnings.warn("Couldn't load compiled cython code. Code will run a bit slower.")


<font color='orange'>
This note demonstrate use MPO-MPS method to construct a SU(2)-symmetric fermionic Z2 QSL.
    
We consider a Z2-QSL ansatz on the square lattice, with a number of $N$ sites,
$$H=\sum_{ij}\sum_{s}t_{ij}c^\dagger_{i,s}c^{}_{i,s}+\sum_{ij}\Delta_{ij}c^\dagger_{i,\uparrow}c^\dagger_{j,\downarrow}-\Delta^*_{ij}c^{}_{i,\uparrow}c^{}_{j,\downarrow}.$$
Only $s$-wave pairings are allowed, namely, $$\Delta_{ij}=\Delta_{ji}.$$
We use the basis of ${\bf c}^\dagger_{s}=\left(c^\dagger_{1,s}, c^\dagger_{2,s}, ..., c^\dagger_{N,s}\right).$ (In this note, we use bold-font to denote a vector of fermions.) Then the above Hamiltonian can be expressed in a compact from:
$$H=\frac{1}{2}\left({\bf c}^\dagger_{\uparrow},{\bf c}^\dagger_{\downarrow},{\bf c}_{\uparrow},{\bf c}_{\downarrow}\right)
\left(\begin{array}{cccc}
t &   &  & \Delta \\
  & t & -\Delta^T & \\
  & -\Delta^* & -t^* & \\
\Delta^\dagger & & & -t^*  
\end{array}\right)
\left(\begin{array}{c} {\bf c}_{\uparrow},{\bf c}_{\downarrow},{\bf c}^\dagger_{\uparrow},{\bf c}^\dagger_{\downarrow} \end{array}\right)+\frac{1}{2}{\rm Tr}[{t}].
$$
The above Hamiltonian can be diagonalized by BdG transformation. The particle-hole symmetry makes the diagonalized Hamiltonian must exhibit the following form:
$$H = \sum_{m=1}^{2N}-\epsilon_{m}\left(d^\dagger_m d_m - d_m d^\dagger_m\right),\quad{}\epsilon_{m}\geq{}0,$$
then the ground state of $H$ can be expressed as 
$$|\Psi\rangle=\prod_{m=1}^{2N}d^\dagger_m|0\rangle_c,$$
with $|0\rangle_c$ the vacumm of $c$-fermion. Therefore, once we obtain the wavefunction encoded in $d$-fermion, we know how to calculate $|\Psi\rangle$. Abstractly, we can always express it as
$${\bf d}^\dagger=\left({\bf c}^\dagger_{\uparrow},{\bf c}^\dagger_{\downarrow}\right)U+\left({\bf c}_{\uparrow},{\bf c}_{\downarrow}\right)V,$$
with $U$ and $V$ diagonalizing $H$. Then, we can use the MPO-MPS method to conver $|\Psi\rangle$ into an MPS, see https://arxiv.org/abs/2001.04611.
</font>

In [2]:
lx = 2;    ly = 2;            # the size of square lattice
bcy = -1;    bcx = 0          # boundary condition for y and x 
t = 1;                        # 1st NN hopping on square lattice
dxy = 0.3;                    # nth pairing on square lattice, with xy symmetry
dxxmyy = 0.2                  # 1st pairing on square lattice, with x^2-y^2 symmetry
params_spin = dict(lx=lx, ly=ly, bcy=bcy, bcx=bcx, dxxmyy=dxxmyy, dxy=dxy)

<font color='orange'>
Below we show an example to compute the Z2 QSL state.

<br> Z2QSLSquare --- the parton mean-field model
<br> u, v        --- the $U$ and $V$ above, WITHOUT SU(2) form
<br> cons_N      --- U(1) conservation of parton. "Z2" means that it is a Z2 QSL. if it is a U1 QSL, use cons_N="N" instead
<br> cons_S      --- so-far we do not implement SU(2) symmetry, therefore it is None
<br> chi_max     --- bond dimension
<br> psi_spin1   --- the obtained MPS
</font>

In [3]:
model = Z2QSLSquare(params_spin)
u, v = model.calc_wannier_state(flag=None)
print(model.eng_real_nonSU2)
params_mpompsz2 = dict(cons_N="Z2", cons_S=None, trunc_params=dict(chi_max=128))
eng_spin1 = MPOMPSZ2(u, v, **params_mpompsz2)
eng_spin1.run()
psi_spin1 = eng_spin1.psi.copy()
psi_spin1.norm = 1
psi_spin1.canonical_form()

calculating nonSU(2) g.s.
t= 1.0 ,  dxy= 0.3 ,  dxxmyy= 0.2
Wannier_Z2 [3.00970966 3.00970966 3.99029034 3.99029034 5.00970966 5.00970966
 5.99029034 5.99029034]
[-1.0198039 -1.0198039 -1.0198039 -1.0198039 -1.0198039 -1.0198039
 -1.0198039 -1.0198039  1.0198039  1.0198039  1.0198039  1.0198039
  1.0198039  1.0198039  1.0198039  1.0198039]
applied the 1-th mode, the fidelity is 1.0, the bond dimension is 2
applied the 2-th mode, the fidelity is 1.0, the bond dimension is 4
applied the 3-th mode, the fidelity is 1.0, the bond dimension is 8
applied the 4-th mode, the fidelity is 1.0, the bond dimension is 16
applied the 5-th mode, the fidelity is 1.0, the bond dimension is 16
applied the 6-th mode, the fidelity is 1.0, the bond dimension is 16
applied the 7-th mode, the fidelity is 1.0, the bond dimension is 16
applied the 8-th mode, the fidelity is 1.0, the bond dimension is 16


<font color='orange'>
Note that $H$ exhibit SU(2) symmetry. It can be revealed by rearrange $H$ as 
    $$H=H_{\uparrow}+H_{\downarrow}+\frac{1}{2}{\rm Tr}[t].$$
Here 
$$H_{\uparrow}=\frac{1}{2}\left({\bf c}^\dagger_{\uparrow},{\bf c}_{\downarrow}\right)
\left(\begin{array}{cc}
t  & \Delta \\
\Delta^\dagger  & -t^*  
\end{array}\right)
\left(\begin{array}{c} {\bf c}_{\uparrow},{\bf c}^\dagger_{\downarrow} \end{array}\right),
$$
and 
$$H_{\downarrow}=\frac{1}{2}\left({\bf c}^\dagger_{\downarrow},{\bf c}_{\uparrow}\right)
\left(\begin{array}{cc}
   t & -\Delta^T  \\
   -\Delta^* & -t^*  \\
\end{array}\right)
\left(\begin{array}{c}{\bf c}_{\downarrow},{\bf c}^\dagger_{\uparrow} \end{array}\right).
$$
$H_{\uparrow}$ is diagonalized by a $2N\times{}2N$ matrix $M_{\uparrow}$ with 
$$M_{\uparrow}=\left(\begin{array}{cc}
   U^\uparrow_{11} & U^\uparrow_{12}  \\
   U^\uparrow_{21}  & U^\uparrow_{22}  \\
\end{array}\right).$$
By $SU(2)$ symmetry, we can denote the matrix $M_{\downarrow}$ diagonalzing $H_{\downarrow}$ as
$$M_{\downarrow}=\left(\begin{array}{cc}
   (U^\uparrow_{22})^* & (U^\uparrow_{21})^*  \\
   (U^\uparrow_{12})^* & (U^\uparrow_{11})^*  \\
\end{array}\right).$$
Indeed, the matrix diagonalzing $H$ reads
$$M=\left(\begin{array}{cccc}
U^\uparrow_{11} &   &  & U^\uparrow_{12} \\
  & (U^\uparrow_{22})^* & (U^\uparrow_{21})^* & \\
  & (U^\uparrow_{12})^* & (U^\uparrow_{11})^* & \\
U^\uparrow_{21} & & & U^\uparrow_{22}  
\end{array}\right)
.
$$

<br> The above special form allowed us to implement $SU(2)$ symmetry for the MPO-MPS method. If we want to implement $SU(2)$ symmetry, we need to (i) construct a 2-spinon mode by arranging a negative-energy mode in $H_{\uparrow}$ and the conjugate mode in $H_{\downarrow}$ in a singlet form and (ii) convert this 2-spinon mode into a MPO. 
<br> In this note, we only consider $U(1)\in{}SU(2)$ symmetry for the $S^z$ conservation. Then we can simply express the ground state as
$$|\Psi\rangle=\prod_{n=1}^{N}\left[({\bf c}^\dagger_{\uparrow}U^{\uparrow}_{11}+{\bf c}_{\downarrow}U^{\uparrow}_{21})_n\right]\prod_{n=1}^{N}\left[({\bf c}^\dagger_{\downarrow}U^{\uparrow,*}_{22}+{\bf c}_{\uparrow}U^{\uparrow,*}_{12})_n\right]|0\rangle_{c}.$$
Below we show an example.


<br> Z2QSLSquare --- the parton mean-field model
<br> flag="SU2"  --- the wannier_state has SU2 symmetry
<br> u, v        --- the $U$ and $V$ above, WITH SU(2) form
<br> cons_N      --- U(1) conservation of parton. "Z2" means that it is a Z2 QSL. if it is a U1 QSL, use cons_N="N" instead
<br> cons_S      --- it can be "2*Sz" or None. In order to benchmark, we use None
<br> chi_max     --- bond dimension
<br> psi_spin2   --- the obtained MPS, with U(1) symmetry w.r.t $S^z$
</font>

In [4]:
model = Z2QSLSquare(params_spin)
u, v = model.calc_wannier_state(flag="SU2")
print(model.eng_real_u)
# Note!! for U(1), use cons_S="2*Sz"
params_mpompsz2 = dict(cons_N="Z2", cons_S=None, trunc_params=dict(chi_max=128))
eng_spin2 = MPOMPSZ2(u, v, **params_mpompsz2)
eng_spin2.run()
psi_spin2 = eng_spin2.psi.copy()
psi_spin2.norm = 1
psi_spin2.canonical_form()

calculating SU(2)-symmetric g.s.
t= 1.0 ,  dxy= 0.3 ,  dxxmyy= 0.2
Ap:  [] 
 Bp:  [] 
 Cp:  [] 
 Dp:  [(0, 0), (1, 1), (2, 2), (3, 3)]
Am:  [] 
 Bm:  [] 
 Cm:  [] 
 Dm:  [(0, 0), (1, 1), (2, 2), (3, 3)]
6.0560865118443576e-15
SU2 eng [-1.0198039 -1.0198039 -1.0198039 -1.0198039 -1.0198039 -1.0198039
 -1.0198039 -1.0198039  1.0198039  1.0198039  1.0198039  1.0198039
  1.0198039  1.0198039  1.0198039  1.0198039]
Wannier_Z2 [2. 2. 3. 3.]
Wannier_Z2 [2. 2. 3. 3.]
[-1.0198039 -1.0198039 -1.0198039 -1.0198039  1.0198039  1.0198039
  1.0198039  1.0198039]
applied the 1-th mode, the fidelity is 1.0, the bond dimension is 2
applied the 2-th mode, the fidelity is 1.0, the bond dimension is 4
applied the 3-th mode, the fidelity is 1.0, the bond dimension is 8
applied the 4-th mode, the fidelity is 1.0, the bond dimension is 16
applied the 5-th mode, the fidelity is 1.0, the bond dimension is 8
applied the 6-th mode, the fidelity is 1.0, the bond dimension is 16
applied the 7-th mode, the fidelity

<font color='orange'>
If we only want to implemnt $U(1)$ symmetry, we can further simplify the MPO-MPS calculation by taking a transformation as 
$${\bf c}_{\downarrow}\Rightarrow{}\tilde{\bf c}^\dagger_{\downarrow}.$$
With this transformatin, the vacuum becomes
    $$|0\rangle_c\Rightarrow{}|0,\downarrow\rangle_c,$$
where ${\bf c}_{\uparrow}|0,\downarrow\rangle_c=0$ and $\tilde{\bf c}_{\uparrow}|0,\downarrow\rangle_c=0.$ Note that $|0,\downarrow\rangle_c$ breaks the SU(2) symmetry. 
<br> It is easy to verify that $\tilde{H}_{\downarrow}$ (the transformed $H_{\downarrow}$) and $H_{\uparrow}$ have the same form, namely, $\tilde{H}_{\downarrow}=H_{\uparrow}$. 
<br> The ground state becomes
$$|\Psi\rangle=\prod_{n=1}^{N}\left[({\bf c}^\dagger_{\uparrow}U^{\uparrow}_{11}+{\bf c}_{\downarrow}U^{\uparrow}_{21})_n\right]|0,\downarrow\rangle_{c}.$$ Therefore, with this transformation, we reduce the number of occupied modes by N.
<br> Below we show an example.
<br>
<br> flag="SU2PH"    ---   SU(2) with particle-hole transformation
<br> cons_S="2*Sz"   ---   U(1) symmetry, it also can be None
</font>

In [5]:
model = Z2QSLSquare(params_spin)
u, v = model.calc_wannier_state(flag="SU2PH")
print(u.shape,v.shape)
params_mpompsz2 = dict(cons_N="Z2", cons_S=None, trunc_params=dict(chi_max=128))
eng_spin3 = MPOMPSZ2(u, v, **params_mpompsz2)
eng_spin3.init_mps(init=[2]*lx*ly)
eng_spin3.run()
psi_spin3 = eng_spin3.psi.copy()
psi_spin3.norm = 1
psi_spin3.canonical_form()

calculating SU(2)-symmetric g.s.
t= 1.0 ,  dxy= 0.3 ,  dxxmyy= 0.2
Ap:  [] 
 Bp:  [] 
 Cp:  [] 
 Dp:  [(0, 0), (1, 1), (2, 2), (3, 3)]
Am:  [] 
 Bm:  [] 
 Cm:  [] 
 Dm:  [(0, 0), (1, 1), (2, 2), (3, 3)]
6.0560865118443576e-15
SU2 eng [-1.0198039 -1.0198039 -1.0198039 -1.0198039 -1.0198039 -1.0198039
 -1.0198039 -1.0198039  1.0198039  1.0198039  1.0198039  1.0198039
  1.0198039  1.0198039  1.0198039  1.0198039]
Wannier_Z2 [2. 2. 3. 3.]
Wannier_Z2 [2. 2. 3. 3.]
(4, 8) (4, 8)
applied the 1-th mode, the fidelity is 1.0, the bond dimension is 2
applied the 2-th mode, the fidelity is 1.0, the bond dimension is 4
applied the 3-th mode, the fidelity is 1.0, the bond dimension is 8
applied the 4-th mode, the fidelity is 1.0, the bond dimension is 16


In [6]:
print(psi_spin2.overlap(psi_spin1), psi_spin2.overlap(psi_spin3), psi_spin3.overlap(psi_spin1))

-1.0000000000000004 -1.0000000000000004 1.0000000000000002
